Title: Finding Value Counts with FILTER
Slug: sql/finding-value-counts-with-filter
Category: SQL
Tags: SELECT, FROM, GROUP BY, ORDER BY, CASE, WHEN, THEN, AS, FILTER, count, sum
Date: 2017-09-03
Modified: 2017-09-03

### Load ipython-sql extension

In [1]:
# The 2 lines below prevent an error message from being displayed when we run %load_ext sql
import warnings
warnings.filterwarnings('ignore')

%load_ext sql
%config SqlMagic.feedback = False

### Connect to the database

In [2]:
%%sql

postgresql://localhost/dvdrental

'Connected: None@dvdrental'

### Basic `count` method

In [3]:
%%sql

SELECT
    f.rental_duration
    , count(f.rental_duration)
FROM
    film f
GROUP BY
    f.rental_duration
ORDER BY
    f.rental_duration ASC

rental_duration,count
3,203
4,203
5,191
6,212
7,191


### A more flexible way with `CASE`
Let's say we want to categorise our values. In older versions of PostgreSQL, we would do something like this:

In [4]:
%%sql

SELECT
    sum(CASE WHEN f.rental_duration < 5 THEN 1 ELSE 0 END) AS under_five
    , sum(CASE WHEN f.rental_duration >= 5 THEN 1 ELSE 0 END) AS five_or_over
FROM
    film f

under_five,five_or_over
406,594


### An even better way
Version 9.4 introduced `FILTER`, letting us write a better, more readable version of the above query.

In [5]:
%%sql

SELECT
    count(*) FILTER(WHERE f.rental_duration < 5) AS under_five
    , count(*) FILTER(WHERE f.rental_duration >= 5) AS five_or_over
FROM
    film f

under_five,five_or_over
406,594


### `FILTER` in practice
With this new function, we can essentially create a pivot table with a single query.

In [6]:
%%sql

SELECT
    f.rental_rate
    , count(*) FILTER(WHERE f.rental_duration < 5) AS under_five
    , count(*) FILTER(WHERE f.rental_duration >= 5) AS five_or_over
FROM
    film f
GROUP BY
    f.rental_rate
ORDER BY
    f.rental_rate ASC

rental_rate,under_five,five_or_over
0.99,150,191
2.99,124,199
4.99,132,204


These examples are based on information in [PostgreSQL: Up and Running](https://www.amazon.co.uk/PostgreSQL-Running-Regina-Obe/dp/1449373194/).